In [1]:
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import bs4
import chromadb.config
from PyPDF2 import PdfReader
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain


import warnings

warnings.filterwarnings("ignore")

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# Load Data

In [11]:
def load_data(
        uploaded_files,
    ):
    TEXT = ""

    # for file in uploaded_files:
    with open(uploaded_files, 'rb') as f:
        # if file.type == "application/pdf":
        pdf_reader = PdfReader(f)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        TEXT += text
        docs = TEXT
    return docs

def embbedings_and_store(docs):
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    persist_directory = "chroma_db"
    
    text_splitter = CharacterTextSplitter(separator=" ", chunk_size=5000, chunk_overlap=1000, length_function=len)
    text_chunks = text_splitter.split_text(docs)
    
    vectorstore = Chroma.from_texts(text_chunks, embedding=embeddings, persist_directory=persist_directory)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    return retriever

In [12]:
docs = load_data(uploaded_files='gobook.pdf')
vector = embbedings_and_store(docs=docs)

In [13]:
vector

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7ff6546cda50>, search_kwargs={'k': 2})

# Create Chain

## model 1

In [14]:
def create_conversational_chain(retriever):
    ### Contextualize question ###
    contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )


    ### Answer question ###
    system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        "{context}"
    )
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    
    store = {}
    
    def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]


    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    return conversational_rag_chain

## model 2

In [15]:
def start_conversation(vector_embeddings):
    # llm = ChatOpenAI()
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True
    )
    conversation = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_embeddings,
        memory=memory
    )

    return conversation

In [21]:
conv = create_conversational_chain(vector)
conv

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBranch(branches=[(RunnableBinding(bound=RunnableLambda(_is_not_async), config={'run_name': 'RunnableWithMessageHistoryInAsyncMode'}), RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
    context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
             | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7ff6546cda50>, search_kwargs={'k': 2}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessa

# Chatting

In [17]:
def conversation_chat(query, chain, history=None):
    result = chain.invoke(
        {"input": query},
        config={
            "configurable": {"session_id": "abc123"}
        },  # constructs a key "abc123" in `store`.
    )["answer"]
    return result

In [18]:
def run_chat(query_input, file, history = None):
    
    # data_loader = load_data(
    #     'dataset/imdb_top_1000.csv'
    # )
    vectorstores = embbedings_and_store(file)
    chain = create_conversational_chain(vectorstores)
    result = conversation_chat(
        query_input, 
        chain=chain, 
        history=history
    )

    return result

# Test

In [23]:
conversation_chat(
    query="tell me just a brief, while class based programming not supported on go, what can we do while we want to make a class ?",
    chain=conv
)

'In Go, class-based programming is not supported. However, you can achieve similar functionality using struct types and methods. Structs in Go can have fields and methods associated with them, allowing you to encapsulate data and behavior together. By defining methods on structs, you can create behavior similar to class methods in other languages.'

In [24]:
conversation_chat(
    query="can you show me how create that ?",
    chain=conv
)

'In Go, you can create a struct with fields and methods to achieve similar functionality to class-based programming. Here is a simple example:\n\n```go\npackage main\n\nimport "fmt"\n\ntype Person struct {\n    Name string\n    Age  int\n}\n\nfunc (p Person) greet() {\n    fmt.Printf("Hello, my name is %s and I am %d years old.\\n", p.Name, p.Age)\n}\n\nfunc main() {\n    p := Person{Name: "Alice", Age: 30}\n    p.greet()\n}\n```\n\nIn this example, we define a `Person` struct with `Name` and `Age` fields. We then define a `greet` method associated with the `Person` struct that prints a greeting message. Finally, in the `main` function, we create an instance of `Person` and call the `greet` method on it.'

In [22]:
result = conv.invoke(
    {
        "question":"tell me just a brief, while class based programming not supported on go, what can we do while we want to make a class ?"
    }
)["answer"]

result

ValueError: Missing keys ['session_id'] in config['configurable'] Expected keys are ['session_id'].When using via .invoke() or .stream(), pass in a config; e.g., chain.invoke({'input': 'foo'}, {'configurable': {'session_id': '[your-value-here]'}})

In [20]:
result = conv.invoke(
    {
        "question":"can you show me how create that ?"
    }
)["answer"]

result

'In Go, you can create a class-like structure using structs and methods. While Go does not have classes like other object-oriented languages, you can achieve similar behavior by defining methods on structs. Here is a simple example:\n\n```go\npackage main\n\nimport "fmt"\n\ntype Person struct {\n    Name string\n    Age  int\n}\n\nfunc (p Person) SayHello() {\n    fmt.Printf("Hello, my name is %s and I am %d years old.\\n", p.Name, p.Age)\n}\n\nfunc main() {\n    p := Person{Name: "Alice", Age: 30}\n    p.SayHello()\n}\n```\n\nIn this example, we define a `Person` struct with `Name` and `Age` fields. We then define a method `SayHello` on the `Person` struct, which allows instances of `Person` to call `SayHello()` to print a greeting with their name and age.\n\nBy attaching methods to structs, you can achieve similar functionality to classes in other languages.'